<h1>Biblioteca</h1>

In [1]:
import pandas as pd
import numpy as np 




<h1>Pré-Processamento</h1>

In [2]:
## Dataframe Origenal
df = pd.read_json('https://jeap.rio.rj.gov.br/dadosAbertosAPI/v2/transporte/veiculos/onibus2')
df

,ordem,linha,latitude,longitude,velocidade,dataHora
0,B32832,298,-22.91219,-43.18789,0,05-27-2022 04:18:29
1,C47499,600,-22.95205,-43.34878,0,05-27-2022 06:06:58
2,D13373,394,-22.88046,-43.35702,0,05-27-2022 06:08:40
3,C47853,693,-22.88024,-43.35835,0,05-27-2022 06:09:30
4,C30262,565,-22.94975,-43.34786,0,05-27-2022 06:35:58
...,...,...,...,...,...,...
3434,A71507,100,-22.97296,-43.18802,37,05-27-2022 11:44:05
3435,B11560,908,-22.87778,-43.32724,33,05-27-2022 11:44:05
3436,B11648,363,-22.90396,-43.28495,0,05-27-2022 11:44:05
3437,B11540,362,-22.83869,-43.25539,21,05-27-2022 11:44:06


In [3]:
##Exportação do dataframe em Python
##df_excel=df
##df_excel.to_excel (r'C:\Users\Gabriel\Desktop\Mestrado\export_dataframe.xlsx', index = False, header=True)

In [6]:
#NOVO DATAFRAME

ordem=df['ordem']
ordemslice=df['ordem'].str.slice(1,4)

df2= pd.DataFrame(list(zip(ordem,ordemslice)),columns=['OrdemOnibus','CodEmpr'])

df2['Empresa'] = [
                  "Transportes Paranapuan S/A" if s>="100" and s<"105" else
                  "Viação Penha Rio Ltda" if s>="105" and s<"115" else
                  "Viação Vila Real S/A" if s>="115" and s<"120" else
                  "Expresso Recreio Ltda" if s>="120" and s<"125" else
                  "Viação Normandy To Triângulo Ltda" if s>="125" and s<"130" else
                  "Transportes Barra Ltda" if s>="130" and s<"170" else 
                  "Auto Viação Palmares Ltda" if s>="170" and s<"255" else
                  "Roviária Âncora Matias" if s>="225" and s<"270" else
                  "Caprichosa Auto Ônibus Ltda" if s>="270" and s<"275" else
                  "Transportes Vila Isabel S/A " if s>="275" and s<"285" else
                  "Viação Ideal S/A" if s>="285" and s<"290" else
                  "Empresa Braso Lisboa" if s>="290" and s<"300" else 
                  "Transportes Futuro Ltda" if s>="300" and s<"310" else 
                  "Viação VG Eireli ME" if s>="310" and s<"325" else 
                  "Viação Pavuenense S/A" if s>="325" and s<"410" else 
                  "Real Auto Ônibus Ltda" if s>="410" and s<"425" else 
                  "Viação Acari S/A" if s>="425" and s<"445" else 
                  "Auto Viação Três Amigos Ltda" if s>="445" and s<"475" else
                  "Viação Redentor Ltda" if s>="475" and s<"480" else
                  "Auto Viação Alpha S/A " if s>="480" and s<"500" else
                  "Tijuquinha - Auto Viação Tijuca S/A" if s>="500" and s<"515" else
                  "Viação Novacap S/A" if s>="515" and s<"535" else
                  "Transportes Campo Grande Ltda" if s>="535" and s<"580" else
                  "Viação Nossa Senhora de Lourdes Ltda" if s>="580" and s<"630" else
                  "Gire Transportes Ltda" if s>="630" and s<"710" else
                  "Viação Verdun S/A" if s>="710" and s<"715" else
                  "Viação Nossa Senhora das Graças Ltda" if s>="715" and s<"720" else
                  "Transurb S/A" if s>="720" and s<"860" else
                  "Auto Viação Jabour Ltda" if s>="860" and s<"870" else
                  "Expresso Pégasso Ltda" if s>="870" and s<"999" 
                   else "BRT Rio S.A"  for s in df2['CodEmpr']] 

#with pd.option_context('display.max_rows', None,
#                     'display.max_columns', None,
#                       'display.precision', 3,
#                     ):
#   print(df2)

df2




,OrdemOnibus,CodEmpr,Empresa
0,B32832,328,Viação Pavuenense S/A
1,C47499,474,Auto Viação Três Amigos Ltda
2,D13373,133,Transportes Barra Ltda
3,C47853,478,Viação Redentor Ltda
4,C30262,302,Transportes Futuro Ltda
...,...,...,...
3434,A71507,715,Viação Nossa Senhora das Graças Ltda
3435,B11560,115,Viação Vila Real S/A
3436,B11648,116,Viação Vila Real S/A
3437,B11540,115,Viação Vila Real S/A
